In [1]:
from BCBio import GFF


In [2]:
# in_handle = open(r"F:\01 科研项目数据-2022\06-基础AI\human.gtf",'r')
# for rec in GFF.parse(in_handle):
#     for feature in rec.features:
#         if feature.type == "region":
#             chromo = feature.qualifiers['ID'][0]
#             chrid = chromo.split(":")[0]
#             chrLen = chromo.split("..")[1]
#             print(chrid,chrLen)

In [3]:
path = r"F:\01 科研项目数据-2022\06-基础AI\human.gtf"

In [4]:
from BCBio import GFF
from collections import Counter

biotype = []

in_handle = open(path,'r')

for rec in GFF.parse(in_handle):
    for feature in rec.features:
        if feature.type == "gene":
            biotype.append(feature.qualifiers['gene_biotype'][0])
            
in_handle.close()
len(biotype)
biotype[0:15]

Counter(biotype)

Counter({'transcribed_unprocessed_pseudogene': 925,
         'unprocessed_pseudogene': 2627,
         'miRNA': 1879,
         'lncRNA': 16887,
         'protein_coding': 19941,
         'processed_pseudogene': 10168,
         'snRNA': 1901,
         'transcribed_processed_pseudogene': 498,
         'misc_RNA': 2211,
         'TEC': 1061,
         'transcribed_unitary_pseudogene': 137,
         'snoRNA': 943,
         'scaRNA': 49,
         'rRNA_pseudogene': 500,
         'unitary_pseudogene': 97,
         'polymorphic_pseudogene': 42,
         'pseudogene': 18,
         'rRNA': 52,
         'IG_V_pseudogene': 188,
         'ribozyme': 8,
         'sRNA': 5,
         'TR_V_gene': 106,
         'TR_V_pseudogene': 33,
         'TR_D_gene': 4,
         'TR_J_gene': 79,
         'TR_C_gene': 6,
         'TR_J_pseudogene': 4,
         'IG_C_gene': 14,
         'IG_C_pseudogene': 9,
         'IG_J_gene': 18,
         'IG_J_pseudogene': 3,
         'IG_D_gene': 37,
         'IG_V_gene': 144,


In [5]:
def get_gene_info(gencode_filename):
    gene_dict = {}
    with open(gencode_filename, "r") as gencode_file, open("gene_dict.csv", "w+") as gene_dict_file:
        effective_genes_num = 0

        for line in gencode_file:  # read line by line

            if line[0:3] != "chr":  # skip useless lines
                continue

            # separate line into 9 parts
            # …… \t …… \t line_info_type \t …… \t …… \t …… \t …… \t …… \t gene_info \t
            line_info = line.split("\t")
            if line_info[2] != "gene":  # skip unimportant lines
                continue

            # eg.gene_id "ENSG00000223972.5"; gene_type "transcribed_unprocessed_pseudogene"; gene_name "DDX11L1"; ……
            gene_info = line_info[8].split(" ")
            gene_id = gene_info[1].split(".")[0][1:]
            gene_name = gene_info[5][1:-2]

            gene_dict[gene_id] = gene_name
            gene_dict_info = gene_id + "," + gene_name + "\n"
            gene_dict_file.write(gene_dict_info)

            effective_genes_num += 1

        print("Effective genes number found in " + gencode_filename + " is " + str(effective_genes_num))
        return gene_dict


def transformENSGid(gencode_filename, tcga_filename):
    gene_dict = get_gene_info(gencode_filename)

    transformedENSGid_num = 0

    with open(tcga_filename, "r") as tcga_data, open("TCGAfpkm_transformed.xls", "w+") as new_tcga_data_file:
        for line in tcga_data:

            if line[0:4] != "ENSG":  # skip useless lines
                new_tcga_data_file.write(line)
                continue

            # separate line into several parts
            line_info = line.split("\t")

            # get gene_id
            gene_id = line_info[0].split(".")[0]

            # transform
            try:
                line_info[0] = gene_dict[gene_id]
            except:
                continue  # skip useless lines

            # generate new line
            new_line = "\t"
            new_line = new_line.join(line_info)
            new_tcga_data_file.write(new_line)

            transformedENSGid_num += 1

        print("Transformed ENSGID in " + tcga_filename + " is " + str(transformedENSGid_num) + ".")


# if __name__ == "__main__":
#     # get file "gencode.v38.chr_patch_hapl_scaff.basic.annotation.gtf" from gencode database
#     # https://www.gencodegenes.org/human/

#     transformENSGid("gencode.v38.chr_patch_hapl_scaff.basic.annotation.gtf", "TCGAfpkm.xls")
#     # transformENSGid("gencode.v38.chr_patch_hapl_scaff.annotation.gtf","TCGAfpkm.xls")


In [6]:
transformENSGid(r"F:\01 科研项目数据-2022\06-基础AI\human.gtf", "TCGAfpkm.xls")

Effective genes number found in F:\01 科研项目数据-2022\06-基础AI\human.gtf is 0


FileNotFoundError: [Errno 2] No such file or directory: 'TCGAfpkm.xls'

In [7]:
gene_dict = get_gene_info(r"F:\01 科研项目数据-2022\06-基础AI\human.gtf")

Effective genes number found in F:\01 科研项目数据-2022\06-基础AI\human.gtf is 0


In [41]:
gencode_filename = r"F:\01 科研项目数据-2022\06-基础AI\human.gtf"

In [42]:
gene_dict = {}

In [10]:
with open(gencode_filename, "r") as gencode_file:
    effective_genes_num = 0
    for line in gencode_file: 
        if line[0:2] == "#!":
            continue
#         print(line)
        line_info = line.split("\t")
        
        if line_info[2] != "gene":  # skip unimportant lines
            continue
        gene_info = line_info[8].split(" ")
        gene_id = gene_info[1]
        gene_name = gene_info[5]
#         print(gene_id, gene_name)
        gene_id=gene_id.replace('"','')
        new_gene_id=gene_id.replace(';','')
        gene_name=gene_name.replace('"','')
        new_gene_name=gene_name.replace(';','')
        gene_dict[new_gene_id] = new_gene_name
        effective_genes_num += 1
#         if effective_genes_num > 10:
#             break
#             # eg.gene_id "ENSG00000223972.5"; gene_type "transcribed_unprocessed_pseudogene"; gene_name "DDX11L1"; ……
#             gene_info = line_info[8].split(" ")
#             gene_id = gene_info[1].split(".")[0][1:]
#             gene_name = gene_info[5][1:-2]
#             print(gene_info)
# #             gene_dict[gene_id] = gene_name
# #             gene_dict_info = gene_id + "," + gene_name + "\n"
# #             gene_dict_file.write(gene_dict_info)

#             effective_genes_num += 1
#             if effective_genes_num > 10:
#                 break

#         print("Effective genes number found in " + gencode_filename + " is " + str(effective_genes_num))

In [11]:
import pandas as pd


In [12]:
df = pd.read_csv(r"F:\01 科研项目数据-2022\06-基础AI\TCGA-COAD-counts\sample0803\Diff_genes_all.csv")

In [14]:
gennames = []
for index, row in df.iterrows():
    
    gend_id = row[1]
    try:
        genname = gene_dict[gend_id]
    except:
        print(gend_id)
        genname = ""
    
    gennames.append(genname)

ENSG00000185087
ENSG00000226958
ENSG00000251164
ENSG00000278309
ENSG00000279487


In [15]:
df['gene_name'] = gennames

In [16]:
df

,Unnamed: 0,gene_id,gene_name
0,0,ENSG00000007171,NOS2
1,1,ENSG00000021826,CPS1
2,2,ENSG00000026559,KCNG1
3,3,ENSG00000057468,MSH4
4,4,ENSG00000058404,CAMK2B
...,...,...,...
423,423,ENSG00000279835,AL121785.1
424,424,ENSG00000279905,AC063926.3
425,425,ENSG00000280228,AC079753.1
426,426,ENSG00000280800,FP671120.7


In [17]:

df.to_csv(r'F:\01 科研项目数据-2022\06-基础AI\Diff_genes_name0804.csv', index=False, float_format="string")

In [1]:
import os
import pandas as pd

In [5]:
files1 = os.listdir(r"G:\基础+AI\切片数据分类\M0")
files2 = os.listdir(r"G:\基础+AI\切片数据分类\M1")
files3 = os.listdir(r"G:\基础+AI\切片数据分类-未用\Mx")
# files1 = os.listdir(r"G:\基础+AI\切片数据分类\M0")

In [7]:
files = files1+files2+files3

In [9]:
len(files)

453

In [10]:
pd.DataFrame(files).to_csv(r"files.csv")

In [1]:
import os

import glob

In [29]:
files = glob.glob(r"Z:\模型算法组\训练数据\zhangkuo\基础AI\DX0810-color-result0816/*/*.png")

In [30]:
len(files)

294418

In [31]:
folds = os.listdir(r"Z:\模型算法组\训练数据\zhangkuo\基础AI\DX0810-color-result0816")

In [32]:
path = r"Z:\模型算法组\训练数据\zhangkuo\基础AI\DX0810-tiles_png"

In [33]:
tiles_files = []
for fold in folds:
    sp = os.path.join(path, fold)
    _files = os.listdir(sp)
    tiles_files += _files

In [34]:
len(tiles_files)

294566

In [35]:
len(folds)

52

In [21]:
282428-294566

-12138

In [37]:
49+266+66+13+52

446

In [40]:
round(1.53, 1)

1.5